In [45]:
import os
import pandas as pd
import plotly
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly as pys
import ipywidgets as widgets
from ipywidgets import Layout

import _global_scripts as gs

## Purpose

Consider removing the crt distance variable that discourages close trips and replace with something that discourages close AND encourages far distances

 - **2.2.1** S-Curve: Discourages close trips and encourages far trips (wCRT & dCRT)
    - 5.8596 / (1 + EXP(-0.6 * (DIST - 10))) - 3
- **2.2.2** S-Curve: Discourages close trips and encourages far trips (dCRT only)
    - 5.8596 / (1 + EXP(-0.6 * (DIST - 10))) - 3
- **2.2.3** S-Curve: Encourages far trips, but less than E2.2.2
    - 1.9880 / (1 + EXP(-0.7 * (DIST - 10)))
- **2.2.4** Piece-Wise: Discourages close trips (original function) and encourage far trips linearly
    - MIN(IF(DIST <= 6, {original_function}, (DIST - 3) * 0.05) , 2)
 

## Inputs

In [46]:
# set whether you want PA or OD analysis
pa_od = 'PA'
pa_od_function = 'pa'

In [47]:
df_tdm_obs = pd.read_csv(f"_data/base_observed_summary_{pa_od_function}.csv").sort_values(by=['Source','station'], ascending=True).reset_index(drop=True)
if(pa_od == 'PA'):
        df_tdm_obs = df_tdm_obs.drop(columns={'Alt_PA','Alt_Direct_PA','Alt_Transfer_PA'})

# read in results for e2.2.1 before recalibration
path_brding_summary_221_b4 = r"_data/E2.2/E2.2.1_node_b4calib.csv"
path_rider_summary_221_b4  = r"_data/E2.2/E2.2.1_link_b4calib.csv"

# read in results for e2.2.2 before recalibration
path_brding_summary_222_b4 = r"_data/E2.2/E2.2.2_node_b4calib.csv"
path_rider_summary_222_b4  = r"_data/E2.2/E2.2.2_link_b4calib.csv"

# read in results for e2.2.3 before recalibration
path_brding_summary_223_b4 = r"_data/E2.2/E2.2.3_node_b4calib.csv"
path_rider_summary_223_b4  = r"_data/E2.2/E2.2.3_link_b4calib.csv"

# read in results for e2.2.4 before recalibration
path_brding_summary_224_b4 = r"_data/E2.2/E2.2.4_node_b4calib.csv"
path_rider_summary_224_b4  = r"_data/E2.2/E2.2.4_link_b4calib.csv"

In [48]:
# read in stations and summarize tdm results
df_stations1 = gs.df_stations[['station','N']]
df_tdm_221_b4 = gs.summarize_tdm_stats(path_brding_summary_221_b4,path_rider_summary_221_b4, df_stations1, 'TDM_2.2.1_b4', pa_od_function)
df_tdm_222_b4 = gs.summarize_tdm_stats(path_brding_summary_222_b4,path_rider_summary_222_b4, df_stations1, 'TDM_2.2.2_b4', pa_od_function)
df_tdm_223_b4 = gs.summarize_tdm_stats(path_brding_summary_223_b4,path_rider_summary_223_b4, df_stations1, 'TDM_2.2.3_b4', pa_od_function)
df_tdm_224_b4 = gs.summarize_tdm_stats(path_brding_summary_224_b4,path_rider_summary_224_b4, df_stations1, 'TDM_2.2.4_b4', pa_od_function)

# concat dataframes
df_tdm_obs_new = pd.concat([
        df_tdm_obs,
        df_tdm_221_b4,
        df_tdm_222_b4,
        df_tdm_223_b4,
        df_tdm_224_b4
    ]).reset_index()
df_tdm_obs_new = df_tdm_obs_new.round(3).fillna(0)
df_tdm_obs_new = df_tdm_obs_new.drop(columns='index')
df_tdm_obs_new = df_tdm_obs_new.loc[:, ~df_tdm_obs_new.columns.str.contains('^Unnamed')]

# rename
sumStats = df_tdm_obs_new.copy()
display(df_tdm_obs_new)

d:\GitHub\WF-TDM-v9x\_scripts\_global_scripts.py:27: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

d:\GitHub\WF-TDM-v9x\_scripts\_global_scripts.py:27: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

d:\GitHub\WF-TDM-v9x\_scripts\_global_scripts.py:27: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

d:\GitHub\WF-TDM-v9x\_scripts\_global_scripts.py:27: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00
...,...,...,...,...,...,...
175,TDM_2.2.4_b4,11-FARMINGTON STATION,walk,136.700,104.35,32.35
176,TDM_2.2.4_b4,12-LAYTON STATION,walk,291.870,113.58,178.29
177,TDM_2.2.4_b4,13-CLEARFIELD STATION,walk,459.460,383.10,76.36
178,TDM_2.2.4_b4,14-ROY STATION,walk,320.020,294.16,25.86


In [49]:
# add a few more columns regarding percentage of boardings in relation to total boardings

# sum by source and station 
station_sum = sumStats.groupby(["Source", "station"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

# add All accessMode
station_sum["AccessMode"] = "All"
sumStats2 = pd.concat([sumStats, station_sum], ignore_index=True) 

accessmode_sum = sumStats2.groupby(["Source", "AccessMode"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

accessmode_sum.rename(columns={
    f"Brd_{pa_od}": f"Source_Brd_{pa_od}", 
    f"Brd_Direct_{pa_od}": f"Source_Brd_Direct_{pa_od}", 
    f"Brd_Transfer_{pa_od}": f"Source_Brd_Transfer_{pa_od}"}, inplace=True)
sumStatsP = sumStats2.merge(accessmode_sum, on=["Source", "AccessMode"], how="left")

sumStatsP[f"Brd_{pa_od}_Perc"]          = sumStatsP[f"Brd_{pa_od}"] / sumStatsP[f"Source_Brd_{pa_od}"]
sumStatsP[f"Brd_Direct_{pa_od}_Perc"]   = sumStatsP[f"Brd_Direct_{pa_od}"] / sumStatsP[f"Source_Brd_Direct_{pa_od}"]
sumStatsP[f"Brd_Transfer_{pa_od}_Perc"] = sumStatsP[f"Brd_Transfer_{pa_od}"] / sumStatsP[f"Source_Brd_Transfer_{pa_od}"]
sumStatsP


,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA,Source_Brd_PA,Source_Brd_Direct_PA,Source_Brd_Transfer_PA,Brd_PA_Perc,Brd_Direct_PA_Perc,Brd_Transfer_PA_Perc
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00,14955.118,0.00,0.0,0.152682,NaN,NaN
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00,5672.892,0.00,0.0,0.166742,NaN,NaN
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00,14955.118,0.00,0.0,0.098630,NaN,NaN
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00,5672.892,0.00,0.0,0.065540,NaN,NaN
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00,14955.118,0.00,0.0,0.095203,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
265,TDM_2.2.4_b4,11-FARMINGTON STATION,All,1245.100,1212.74,32.36,36622.470,33880.27,2742.2,0.033998,0.035795,0.011801
266,TDM_2.2.4_b4,12-LAYTON STATION,All,2625.630,2447.31,178.32,36622.470,33880.27,2742.2,0.071695,0.072234,0.065028
267,TDM_2.2.4_b4,13-CLEARFIELD STATION,All,2280.610,2204.26,76.35,36622.470,33880.27,2742.2,0.062274,0.065060,0.027843
268,TDM_2.2.4_b4,14-ROY STATION,All,3646.870,3621.15,25.72,36622.470,33880.27,2742.2,0.099580,0.106881,0.009379


In [50]:
def plotit(variable, access_mode):
    output.clear_output()  # Clear previous output before displaying new content
    global firstTime
    if firstTime:
    
        filtered_data = sumStatsP[sumStatsP['AccessMode'] == access_mode]
            
        # Create histogram
        fig = px.histogram(
            filtered_data, 
            x="station", 
            y=variable, 
            text_auto='.2s',
            color='Source', 
            barmode='group',
            height=400
        )
        fig.update_layout(
            xaxis_title="Station Name",
            yaxis_title=str(variable),
            legend_title="Model Version"
        )
        
        # Display the plot
        fig.show()
    
    else:
        firstTime = True

In [54]:
lstValues = list([
    f'Brd_{pa_od}',
    f'Brd_Direct_{pa_od}',
    f'Brd_Transfer_{pa_od}',
    f'Brd_{pa_od}_Perc',
    f'Brd_Direct_{pa_od}_Perc',
    f'Brd_Transfer_{pa_od}_Perc'])
accessModeOptions = ['drive', 'walk', 'All']

selectValues = widgets.Select(options=lstValues, value=(f'Brd_{pa_od}' ), description = 'Select Variable')
selectAccessMode = widgets.Dropdown(options=accessModeOptions, value='All', description='Access Mode')

# Set up a global variable to track whether the widgets have been changed
firstTime = False

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectValues, selectAccessMode])

# create interactive widget
interactive_output = widgets.interactive_output(plotit, {'variable':selectValues, 'access_mode': selectAccessMode})

display(hbox)
display(interactive_output)
display(output)

Output()

Output()

## Conclusions

- the distance on crt utility variable doesn't play a large role in utility equation
- all functions shared a positive utility given for longer trips on crt -- this resulted in all scenarios slightly improving that of provo central station. Unfortunately, the effects were negated and shared through the system, not even impacting Provo more than other stations. Therefore, we conclude the ran scenarios do not have a significant role in solving the problem
- we must still analyze SANDAG's equation